In [1]:
import pandas as pd
import numpy as np
from skimage import io
import matplotlib.pyplot as plt
import os
from skimage.transform import rescale
from skimage.color import rgb2gray, gray2rgb
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader, Subset, ConcatDataset, random_split
from torchvision import transforms, utils
import random
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable
import time
import cv2
from os.path import expanduser

In [2]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
plt.ion()

In [3]:
dataframe=pd.read_csv('list.csv')

In [4]:
def image_transform(image):
    #Construct the mask
    mask=np.zeros((128,128,3))
    mask[48:80,48:80]=1
    
    image=image/np.amax(image) #All entries of image between 0 and 1
    image=rescale(image,0.5,anti_aliasing=True,multichannel=True) # Rescale image to 128*128*3
    if (image.shape==mask.shape):
        i=np.multiply(image,np.ones(mask.shape)-mask) #Find the unmasked portion
        mu=np.zeros((128,128,3))
        mu[:,:,0]=np.mean(i[:,:,0]) #Calculate the mean over unmasked portion
        mu[:,:,1]=np.mean(i[:,:,1])
        mu[:,:,2]=np.mean(i[:,:,2])
        im=mu*mask+i #Set the outer pixel of channel to mean value calculated above
        gray_img = rgb2gray(mask) 
        ip=np.dstack((im,gray_img)) #Stack final image with the mask
        if (image is not None and ip is not None and mask is not None):
            return (image,ip,mask)

In [5]:
class PlacesDataset(Dataset):
    def __init__(self,df,img_dir,transform=None):
        self.df=df
        self.img_dir=img_dir
        self.transform=transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        img_path=os.path.join(self.img_dir,'{}'.format(self.df.loc[idx]['image']))
        if(img_path is not None):
            image=io.imread(img_path)
            if (image is not None):
                image,tr_image,mask=image_transform(image)
                sample={'orig_image':image,'new_image':tr_image,'mask':mask}
                if self.transform:
                    sample['orig_image']=self.transform(sample['orig_image'])
                    sample['new_image']=self.transform(sample['new_image'])
                    sample['mask']=self.transform(sample['mask'])
        
                return sample

In [6]:
dataset=PlacesDataset(dataframe,img_dir='images/test_256/',transform=transforms.Compose([transforms.ToTensor()]))

In [7]:
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [8]:
random.seed(0)
num_train=len(dataset)
indices=list(range(num_train))
test_idx = indices[-210:]
train_idx = list(set(indices) - set(test_idx))
test_data = Subset(dataset,test_idx)
train_data=Subset(dataset,train_idx)
validation_data,test_data=random_split(test_data,(20,190))
a=len(train_data)-75000
train_phase_1,train_phase_2,train_phase_3,extra=random_split(train_data,(70000,5000,288900,28))

In [9]:
train_loader_p1 = torch.utils.data.DataLoader(train_phase_1,batch_size=100)
train_loader_p2 = torch.utils.data.DataLoader(train_phase_2,batch_size=100)
train_loader_p3 = torch.utils.data.DataLoader(train_phase_3,batch_size=100)
validation_loader = torch.utils.data.DataLoader(validation_data,batch_size=1)
test_loader = torch.utils.data.DataLoader(test_data,batch_size=1)

In [10]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.conv1=nn.Conv2d(in_channels=4,out_channels=64,kernel_size=3,dilation=1,stride=1,padding=1)
        self.norm1=nn.BatchNorm2d(num_features=64)
        self.conv2=nn.Conv2d(in_channels=64,out_channels=128,kernel_size=2,dilation=1,stride=2,padding=0)
        self.norm2=nn.BatchNorm2d(num_features=128)
        self.conv3=nn.Conv2d(in_channels=128,out_channels=256,kernel_size=2,dilation=1,stride=2,padding=0)
        self.norm3=nn.BatchNorm2d(num_features=256)
        self.conv4=nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,dilation=1,stride=1,padding=1)
        self.norm4=nn.BatchNorm2d(num_features=256)
        self.conv5=nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,dilation=2,stride=1,padding=2)
        self.norm5=nn.BatchNorm2d(num_features=256)
        self.conv6=nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,dilation=4,stride=1,padding=4)
        self.norm6=nn.BatchNorm2d(num_features=256)
        self.conv7=nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,dilation=4,stride=1,padding=4)
        self.norm7=nn.BatchNorm2d(num_features=256)
        self.conv8=nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,dilation=8,stride=1,padding=8)
        self.norm8=nn.BatchNorm2d(num_features=256)
        self.conv9=nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,dilation=16,stride=1,padding=16)
        self.norm9=nn.BatchNorm2d(num_features=256)
        self.conv10=nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,dilation=1,stride=1,padding=1)
        self.norm10=nn.BatchNorm2d(num_features=256)
        self.conv11=nn.ConvTranspose2d(in_channels=256,out_channels=128,kernel_size=4,dilation=1,stride=2,padding=1)
        self.norm11=nn.BatchNorm2d(num_features=128)
        self.conv12=nn.Conv2d(in_channels=128,out_channels=128,kernel_size=3,dilation=1,stride=1,padding=1)
        self.norm12=nn.BatchNorm2d(num_features=128)
        self.conv13=nn.ConvTranspose2d(in_channels=128,out_channels=64,kernel_size=4,dilation=1,stride=2,padding=1)
        self.norm13=nn.BatchNorm2d(num_features=64)
        self.conv14=nn.Conv2d(in_channels=64,out_channels=32,kernel_size=3,dilation=1,stride=1,padding=1)
        self.norm14=nn.BatchNorm2d(num_features=32)
        self.conv15=nn.Conv2d(in_channels=32,out_channels=3,kernel_size=3,dilation=1,stride=1,padding=1)
        self.relu=nn.ReLU()
        self.sigm=nn.Sigmoid()
        
    def forward(self,x):
        x=x.type(torch.cuda.FloatTensor)
        out=self.conv1(x)
        out=self.relu(out)
        out=self.norm1(out)
        out=self.conv2(out)
        out=self.relu(out)
        out=self.norm2(out)
        out=self.conv3(out)
        out=self.relu(out)
        out=self.norm3(out)
        out=self.conv4(out)
        out=self.relu(out)
        out=self.norm4(out)
        out=self.conv5(out)
        out=self.relu(out)
        out=self.norm5(out)
        out=self.conv6(out)
        out=self.relu(out)
        out=self.norm6(out)
        out=self.conv7(out)
        out=self.relu(out)
        out=self.norm7(out)
        out=self.conv8(out)
        out=self.relu(out)
        out=self.norm8(out)
        out=self.conv9(out)
        out=self.relu(out)
        out=self.norm9(out)
        out=self.conv10(out)
        out=self.relu(out)
        out=self.norm10(out)
        out=self.conv11(out)
        out=self.relu(out)
        out=self.norm11(out)
        out=self.conv12(out)
        out=self.relu(out)
        out=self.norm12(out)
        out=self.conv13(out)
        out=self.relu(out)
        out=self.norm13(out)
        out=self.conv14(out)
        out=self.relu(out)
        out=self.norm14(out)
        
        out=self.conv15(out)
        out=self.sigm(out)
        
        return out

In [11]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv1g=nn.Conv2d(in_channels=3,out_channels=32,kernel_size=5,dilation=1,stride=2)
        self.norm1g=nn.BatchNorm2d(num_features=32)
        self.conv2g=nn.Conv2d(in_channels=32,out_channels=64,kernel_size=5,dilation=1,stride=2)
        self.norm2g=nn.BatchNorm2d(num_features=64)
        self.conv3g=nn.Conv2d(in_channels=64,out_channels=128,kernel_size=5,dilation=1,stride=2)
        self.norm3g=nn.BatchNorm2d(num_features=128)
        self.conv4g=nn.Conv2d(in_channels=128,out_channels=256,kernel_size=5,dilation=1,stride=2)
        self.norm4g=nn.BatchNorm2d(num_features=256)
        self.conv5g=nn.Conv2d(in_channels=256,out_channels=512,kernel_size=5,dilation=1,stride=2)
        
        self.conv1l=nn.Conv2d(in_channels=3,out_channels=32,kernel_size=5,dilation=1,stride=2)
        self.norm1l=nn.BatchNorm2d(num_features=32)
        self.conv2l=nn.Conv2d(in_channels=32,out_channels=64,kernel_size=5,dilation=1,stride=2)
        self.norm2l=nn.BatchNorm2d(num_features=64)
        self.conv3l=nn.Conv2d(in_channels=64,out_channels=128,kernel_size=5,dilation=1,stride=2)
        self.norm3l=nn.BatchNorm2d(num_features=128)
        self.conv4l=nn.Conv2d(in_channels=128,out_channels=256,kernel_size=5,dilation=1,stride=2)
        
        self.fc=nn.Linear(768,1)
        self.relu=nn.ReLU()
        self.sigm=nn.Sigmoid()
        
    def forward(self,x):
        x=x.type(torch.cuda.FloatTensor)
        out_g=self.conv1g(x)
        out_g=self.relu(out_g)
        out_g=self.norm1g(out_g)
        out_g=self.conv2g(out_g)
        out_g=self.relu(out_g)
        out_g=self.norm2g(out_g)
        out_g=self.conv3g(out_g)
        out_g=self.relu(out_g)
        out_g=self.norm3g(out_g)
        out_g=self.conv4g(out_g)
        out_g=self.relu(out_g)
        out_g=self.norm4g(out_g)
        out_g=self.conv5g(out_g)
        out_g=self.relu(out_g)
        out_g=out_g.view(-1,512)
        
        
        y=x[:,:,32:96,32:96]
        out_l=self.conv1l(y)
        out_l=self.relu(out_l)
        out_l=self.norm1l(out_l)
        out_l=self.conv2l(out_l)
        out_l=self.relu(out_l)
        out_l=self.norm2l(out_l)
        out_l=self.conv3l(out_l)
        out_l=self.relu(out_l)
        out_l=self.norm3l(out_l)
        out_l=self.conv4l(out_l)
        out_l=self.relu(out_l)
        out_l=out_l.view(-1,256)
        
        out=torch.cat((out_g,out_l),1)
        out=self.relu(out)
        out=self.fc(out)
        out=self.sigm(out)
        
        return out

In [12]:
def ones_target(size):
    '''
    Tensor containing ones, with shape = size
    '''
    data = Variable(torch.ones(size, 1))
    return data

def zeros_target(size):
    '''
    Tensor containing zeros, with shape = size
    '''
    data = Variable(torch.zeros(size, 1))
    return data

In [13]:
def evaluate(generator,validation_loader,g_criterion,epoch,num_epochs,epochs):
    running_loss=0
    generator.eval()
    
    home = expanduser('~')
    os.mkdir(home+'/Ashwin/Outpainting/images/val/val_{}'.format(epoch+1))
    
    with torch.no_grad():
        for i,(sample) in enumerate(validation_loader):
            i_p=Variable(sample['new_image']).to(device)
            i_n=Variable(sample['orig_image']).to(device)
            mask=Variable(sample['mask']).to(device)
            mask=mask.type(torch.cuda.FloatTensor)
            i_n=i_n.type(torch.cuda.FloatTensor)
            i_o=generator(i_p)
            loss=g_criterion(torch.mul(i_o,mask),torch.mul(i_n,mask))
            running_loss+=loss.item()
            io.imsave('images/val/val_{}/{}_src.jpg'.format(epoch+1,i),np.transpose(i_o[0,:,:,:].cpu().numpy(), (1, 2, 0)))
            io.imsave('images/val/val_{}/{}_dst.jpg'.format(epoch+1,i),np.transpose(i_p[0,0:3,:,:].cpu().numpy(), (1, 2, 0)))
            io.imsave('images/val/val_{}/{}_org.jpg'.format(epoch+1,i),np.transpose(i_n[0,:,:,:].cpu().numpy(), (1, 2, 0)))
    
    print ('Epoch: [{}/{}] | Validation Loss: {}'.format(epoch+1-epochs, num_epochs,running_loss/20))

In [14]:
def train_both(model_gen,model_dis,train_loader,d_criterion,g_criterion,d_optimizer,g_optimizer,epoch,num_epochs,epochs,alpha,rl_d_real,rl_d_fake,rl_gen):
    running_loss_d_real=0
    running_loss_d_fake=0
    running_loss_g=0
    generator.train()
    discriminator.train()
    for i,(sample) in enumerate(train_loader):
        i_p=Variable(sample['new_image']).to(device)
        i_n=Variable(sample['orig_image']).to(device)
        i_n=i_n.type(torch.cuda.FloatTensor)
        mask=Variable(sample['mask']).to(device)
        mask=mask.type(torch.cuda.FloatTensor)
        d_optimizer.zero_grad()
        i_o=generator(i_p).detach()
        i_p=i_p.type(torch.cuda.FloatTensor)
        i_p[:,:,48:80,48:80]=0
        img=torch.mul(i_o,mask)+i_p[:,0:3,:,:]
        
        
        #Discriminator Training
        
        d_i_o=discriminator(img)
        d_i_n=discriminator(i_n)
        loss_real=d_criterion(d_i_n,ones_target(100).to(device))
        loss_real.backward()
        rl_d_real.append(loss_real.item())
        
        loss_fake=d_criterion(d_i_o,zeros_target(100).to(device))
        loss_fake.backward()
        rl_d_fake.append(loss_fake.item())
        
        d_optimizer.step()
        
        running_loss_d_real+=loss_real.item()
        running_loss_d_fake+=loss_fake.item()
        
        #Generator training
        
        g_optimizer.zero_grad()
        i_p=Variable(sample['new_image']).to(device)
        i_n=Variable(sample['orig_image']).to(device)
        i_n=i_n.type(torch.cuda.FloatTensor)
        mask=Variable(sample['mask']).to(device)
        mask=mask.type(torch.cuda.FloatTensor)
        i_o=generator(i_p)
        i_p=i_p.type(torch.cuda.FloatTensor)
        i_p[:,:,48:80,48:80]=0
        img=torch.mul(i_o,mask)+i_p[:,0:3,:,:]
        d_i_o=discriminator(img).detach()
        loss_g=g_criterion(torch.mul(i_o,mask),torch.mul(i_n,mask)) + alpha*d_criterion(d_i_o,zeros_target(100).to(device))
        loss_g.backward()
        g_optimizer.step()
        d_optimizer.step()
        running_loss_g+=loss_g.item()
        rl_gen.append(loss_g)
        if((i+1)%289==0):
            print('Epoch: [{}/{}] | Step: [{}/{}] | Gen_Loss: {} | Dis_Loss_real: {} | Dis_Loss_fake: {}'.format(epoch+1-epochs, num_epochs, int((i+1)/289),10 , round(running_loss_g/289,4), round(running_loss_d_real/289,6), round(running_loss_d_fake/289,6)))
            running_loss_g=0
            running_loss_d=0

In [15]:
generator = Generator()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
generator=generator.to(device)
g_criterion = nn.MSELoss()
g_optimizer = torch.optim.Adam(generator.parameters())

cuda:0


In [16]:
discriminator=Discriminator()
discriminator=discriminator.to(device)
d_criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(discriminator.parameters())

In [23]:
checkpoint = torch.load('GAN/gen_model_9.pth')
generator.load_state_dict(checkpoint['model_state_dict'])
g_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epochs_gen = checkpoint['epoch']
rl_gen = checkpoint['loss']

generator.eval()

Generator(
  (conv1): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 128, kernel_size=(2, 2), stride=(2, 2))
  (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(128, 256, kernel_size=(2, 2), stride=(2, 2))
  (norm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2))
  (norm5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(4, 4), dilation=(4, 4))
  (norm6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affin

In [24]:
checkpoint = torch.load('GAN/dis_model_9.pth')
discriminator.load_state_dict(checkpoint['model_state_dict'])
d_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epochs_dis = checkpoint['epoch']
rl_dis_real = checkpoint['loss_real']
rl_dis_fake = checkpoint['loss_fake']

discriminator.eval()

Discriminator(
  (conv1g): Conv2d(3, 32, kernel_size=(5, 5), stride=(2, 2))
  (norm1g): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2g): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2))
  (norm2g): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3g): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2))
  (norm3g): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4g): Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2))
  (norm4g): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5g): Conv2d(256, 512, kernel_size=(5, 5), stride=(2, 2))
  (conv1l): Conv2d(3, 32, kernel_size=(5, 5), stride=(2, 2))
  (norm1l): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2l): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2))
  (norm2l): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

In [ ]:
epochs=9
num_epochs=3
#rl_dis_real=[]
#rl_dis_fake=[]
#rl_gen=[]
for epoch in range(epochs,num_epochs+epochs):
    train_both(generator,discriminator,train_loader_p3,d_criterion,g_criterion,d_optimizer,g_optimizer,epoch,num_epochs,epochs,0.0004,rl_dis_real,rl_dis_fake,rl_gen)
    
    torch.save({
            'epoch': int(epoch+1),
            'model_state_dict': discriminator.state_dict(),
            'optimizer_state_dict': d_optimizer.state_dict(),
            'loss_real': rl_dis_real,
            'loss_fake': rl_dis_real,
            },'GAN/dis_model_{}.pth'.format(epoch+1))
    torch.save({
            'epoch': int(epoch+1),
            'model_state_dict': generator.state_dict(),
            'optimizer_state_dict': g_optimizer.state_dict(),
            'loss': rl_gen,
            },'GAN/gen_model_{}.pth'.format(epoch+1))
    
    evaluate(generator,validation_loader,g_criterion,epoch,num_epochs,epochs)
    print ('%---Saving the model---%')

/home/ubuntu/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Epoch: [1/3] | Step: [1/10] | Gen_Loss: 0.0016 | Dis_Loss_real: 0.020431 | Dis_Loss_fake: 0.052293
Epoch: [1/3] | Step: [2/10] | Gen_Loss: 0.0016 | Dis_Loss_real: 0.020431 | Dis_Loss_fake: 0.052473
Epoch: [1/3] | Step: [3/10] | Gen_Loss: 0.0016 | Dis_Loss_real: 0.020435 | Dis_Loss_fake: 0.052493
Epoch: [1/3] | Step: [4/10] | Gen_Loss: 0.0016 | Dis_Loss_real: 0.020444 | Dis_Loss_fake: 0.05251
Epoch: [1/3] | Step: [5/10] | Gen_Loss: 0.0016 | Dis_Loss_real: 0.020445 | Dis_Loss_fake: 0.052523
Epoch: [1/3] | Step: [6/10] | Gen_Loss: 0.0016 | Dis_Loss_real: 0.020526 | Dis_Loss_fake: 0.052761
Epoch: [1/3] | Step: [7/10] | Gen_Loss: 0.0016 | Dis_Loss_real: 0.020526 | Dis_Loss_fake: 0.052769
Epoch: [1/3] | Step: [8/10] | Gen_Loss: 0.0016 | Dis_Loss_real: 0.020526 | Dis_Loss_fake: 0.052773
Epoch: [1/3] | Step: [9/10] | Gen_Loss: 0.0016 | Dis_Loss_real: 0.020526 | Dis_Loss_fake: 0.052775


/home/ubuntu/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:130: UserWarning: Possible precision loss when converting from float32 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:130: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


Epoch: [1/3] | Validation Loss: 0.008324115863069892
%---Saving the model---%
Epoch: [2/3] | Step: [1/10] | Gen_Loss: 0.0016 | Dis_Loss_real: 0.00201 | Dis_Loss_fake: 0.000815
Epoch: [2/3] | Step: [2/10] | Gen_Loss: 0.0016 | Dis_Loss_real: 0.002292 | Dis_Loss_fake: 0.000854
Epoch: [2/3] | Step: [3/10] | Gen_Loss: 0.0016 | Dis_Loss_real: 0.00638 | Dis_Loss_fake: 0.013841
Epoch: [2/3] | Step: [4/10] | Gen_Loss: 0.0016 | Dis_Loss_real: 0.006628 | Dis_Loss_fake: 0.013867
Epoch: [2/3] | Step: [5/10] | Gen_Loss: 0.0016 | Dis_Loss_real: 0.006628 | Dis_Loss_fake: 0.014835
Epoch: [2/3] | Step: [6/10] | Gen_Loss: 0.0016 | Dis_Loss_real: 0.006628 | Dis_Loss_fake: 0.015034
Epoch: [2/3] | Step: [7/10] | Gen_Loss: 0.0016 | Dis_Loss_real: 0.006639 | Dis_Loss_fake: 0.015991
Epoch: [2/3] | Step: [8/10] | Gen_Loss: 0.0016 | Dis_Loss_real: 0.008943 | Dis_Loss_fake: 0.016661
